In [1]:
import sqlite3


In [11]:
# create connection with db
conn = sqlite3.connect('emaildb.sqlite')

# create handler for db
cur = conn.cursor()


In [16]:
# cur.execute('''
# DROP TABLE IF EXISTS Counts''')

In [12]:
# create table Counts
cur.execute('''
CREATE TABLE Counts (org TEXT, count INTERGER)''')

In [157]:
# read from mbox.txt the raw data, given by the course
# write in csv only emails
fname = 'mbox.txt'
fh = open(fname)

# create a new csv in order to config and analyze the raw data
fw = open("demofile3.csv", "w")

for line in fh:
    if not line.startswith('From: '): continue
    pieces = line.split()
    email = pieces[1].split("@") # splitting the domain from emails in order he number of
                                 #email messages per organization
    
    fw.write(email[1])
    fw.write("\n")


In [59]:
# close the r/w streams
fh.close()
fw.close()

In [60]:
import pandas as pd

In [116]:
# create a pandas Dataframe to count mails per organization
# and push the data to a db
df = pd.read_csv("demofile3.csv", names=['emails'])
df.head()

,emails
0,uct.ac.za
1,media.berkeley.edu
2,umich.edu
3,iupui.edu
4,umich.edu


In [118]:
counts = df.groupby(['emails']).size()


In [125]:
counts.to_csv("counts.csv", header=False)

In [139]:
fname = 'counts.csv'
fh = open(fname)

for line in fh:
    p = line.split(",")
    row = cur.fetchone()
    if row is None:
        cur.execute('''INSERT INTO Counts (org, count)
                VALUES (?, ?)''', (p[0], p[1]))
    conn.commit()

In [147]:
sqlstr = 'SELECT * FROM Counts ORDER BY count DESC'
row = cur.fetchone()
for row in cur.execute(sqlstr):
    print(row)



('iupui.edu', 536)
('umich.edu', 491)
('indiana.edu', 178)
('caret.cam.ac.uk', 157)
('vt.edu', 110)
('uct.ac.za', 96)
('media.berkeley.edu', 56)
('ufp.pt', 28)
('gmail.com', 25)
('et.gatech.edu', 17)
('txstate.edu', 17)
('whitman.edu', 17)
('bu.edu', 14)
('lancaster.ac.uk', 14)
('stanford.edu', 12)
('loi.nl', 9)
('unicon.net', 9)
('rsmart.com', 8)
('fhda.edu', 1)
('ucdavis.edu', 1)
('utoronto.ca', 1)


In [148]:
cur.close()
